# Benchmarking demosaicing method on test dataset shot by C2B camera

Given 10 scenes/objects shot in 4 different projector patterns and 3 frames $(S=4, F=3)$. We have a total of $10*4=40$ images. 

The ground truth image for each scene is the __left hand side__ of the following expression. The __left hand side__ of the following image can be thought of as __full-resolution demosaiced__ measurement of the two bucket assuming the camera is operating under
+ The $S$ patterns / frame
+ tiling and correspondence encoded in the code tensor $\mathcal{C}_1$, i.e. code tensor slice for 1 frame

The __right hand side__ image is a matrix multiple of 
+ full-resolution image of the $S$ patterns
+ code tensor for bucket-{1,2}


$$
\newcommand{\hbar}{\rule[.5ex]{2.5ex}{0.5pt}}
\newcommand{\bi}{\mathbf{i}}
\underbrace{
    \begin{pmatrix} 
    \hbar & \bi^1 & \hbar & \hat{\bi}^1 & \hbar 
    \\ &&\vdots&& \\ 
    \hbar & \bi^P & \hbar & \hat{\bi}^P & \hbar
    \end{pmatrix}
}_{P \times 2F}
=
\underbrace{
    \begin{pmatrix} 
    \hbar & i^1 & \hbar 
    \\ &\vdots& \\ 
    \hbar & i^P & \hbar
    \end{pmatrix}
}_{P \times S}
\underbrace{
    \begin{pmatrix}
    \mathbf{C}^p \\
    \mathbf{\overline{C}}^p
    \end{pmatrix}^T
}_{S \times 2F}
$$

where $P$ is the number of pixels in one image and the code matrix $\mathbf{C}^p$ is same for all pixels

$$
\mathbf{C}^p
=
\begin{pmatrix}
    1 & 1 & 0 & 0 \\
    1 & 0 & 1 & 0 \\
    1 & 0 & 0 & 1 \\
\end{pmatrix}
$$

In the end, we obtain $2F=20$ full-resolution images, 2 bucket images for each of 10 scenes for demosaicing comparison. We downsample them according to bayer pattern, and run demosaicing algorithms over the 20 images


In [ ]:
import sys; sys.path.append('..')
import demosaicing as dm
import importlib; importlib.reload(dm)

import os
import re
import cv2 as cv
import numpy as np
from scipy.linalg import hadamard
import glob
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

In [ ]:
topdir = "../data/exp60"
imgdir = os.path.join(topdir, 'organized')
S = 4
F = 3
n_scenes = 10
test_scene = 'bowl'
imgsize = 176, 288
P = imgsize[0]*imgsize[1]

In [ ]:
imgpaths = sorted(glob.glob(imgdir+'/*.png'))
assert(len(imgpaths) == 10*S)

# {scene: (176, 288, S)}
scenes = {}

for imgpath in imgpaths:
    m = re.search(r'([a-z]*)_(\d).png',imgpath)
    assert(m is not None)
    scene = m[1]
    pattern_id = int(m[2])
    
    img = cv.imread(imgpath,cv.IMREAD_GRAYSCALE) / 255
    if scene not in scenes:
        scenes[scene] = np.zeros((*np.shape(img),S))
    scenes[scene][:,:,pattern_id] = img
    
scenes.keys()

In [ ]:
dm.show_grayscale(np.hstack([scenes[test_scene][:,:,i] for i in range(S)]))

In [ ]:
C = (hadamard(S)+1)/2
C = C[1:,:]
W = np.vstack((C,1-C))
W

In [ ]:
# {c2bimgs: (2, 176, 288, 3)}
c2bimgs = {}
test_imgs = []

for scene,img in scenes.items():    
    c2bimg  = np.reshape(img,(-1,4)) @ W.T
    bucket1 = np.reshape(c2bimg[:,0:F],   (*imgsize,F))
    bucket2 = np.reshape(c2bimg[:,F:2*F], (*imgsize,F))

    assert(np.max(bucket1) <= 2 and np.min(bucket1) >= 0)
    assert(np.max(bucket2) <= 2 and np.min(bucket2) >= 0)
    print(f'{scene}\t({np.min(bucket1)}, {np.max(bucket1)})')
    
    c2bimgs[scene] = np.zeros((2,*imgsize,F), dtype=np.uint8)
    c2bimgs[scene][0,:,:,:] = np.clip(bucket1*255/2,0,255)
    c2bimgs[scene][1,:,:,:] = np.clip(bucket2*255/2,0,255)
    
    test_imgs += [c2bimgs[scene][0,:,:,:], c2bimgs[scene][1,:,:,:]]
    
len(test_imgs)

In [ ]:
dm.show_grayscale(np.vstack((
    np.hstack([c2bimgs[test_scene][0,:,:,i] for i in range(3)]),
    np.hstack([c2bimgs[test_scene][1,:,:,i] for i in range(3)]),
)))

In [ ]:
demosaic_algos = [
    ('opencv_bilinear',     lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR)),
    ('opencv_vng',          lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_VNG)),
    ('opencv_edge_aware',   lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_EA)),
    ('bilinear',            dm.demosaic_bilinear),
    ('smooth_hue',          dm.demosaic_smooth_hue),
    ('median_filter',       dm.demosaic_median_filter),   
    ('laplacian_corrected', dm.demosaic_laplacian_corrected)
]

history = {}
print(f"{'method':>20}\t{'mean_pnsr':>20}\t{'median_psnr':>20}")
for algo, f in demosaic_algos:
    history[algo] = dm.validate_imgs(f, test_imgs)
    print(f"{algo:>20}\t{np.mean(history[algo]['psnr']):>20.3f}\t{np.median(history[algo]['psnr']):>20.3f}")